# Problemas de generalización

clase 11, 24 de febrero de 2021

---
Cuando se crea un modelo esperamos que este haya generalizado de manera correcta, sin embargo, es posible que no tenga el rendimiento o propiedades de generalización esperadas. 

Para producción.

Entre los principales problemas de generalización se encuentran los siguientes:

* Datos: fijos, no nos hemos preguntado si son suficientes o representativos de la población de dónde vienen.
* Ambiente: la interacción de los usuarios que puede llevar a un sesgo en el conjunto de prueba.
* Quisiéramos tener buenas capacidades predictivas que no observamos/usamos para entrenar.


## 1 Cambio en la distribución de los datos

Entrenamiento recibimos atributos y etiquetas para un conjunto de datos $(X^{(i)},y^{(i)})_{\ i=1,..,n} \sim \Pi(x,y)$ y establecemos un modelo $h(x^{(i)}) \approx y^{(i)}$ que sea capaz de regresar una buena predicción, la cual busca minimizar una función de pérdida $L_{n}(\theta) = \frac{1}{n} \sum_{i=1}^{n} \mathscr{l}^{(i)}(\hat{y}(\theta),y)$.

La solución $\theta^{s} = argmin (L_{n} | \theta)$.


Prueba (Validación) tenemos otro conjunto de datos: $(X_{p}^{(i)},y_{p}^{(i)}) \sim^{iid} \Pi(X,y), i = 1,...,m$.

Lo que hacemos es evaluar la función de pérdida con este nuevo conjunto de entrenamiento con la solución que encontramos en el entrenamiento. 

$L_{m}(\theta^{s}) = \frac{1}{m} \sum_{j=1}^{m} \mathscr{l}^{(j)}\ (\hat{y}(\theta),y)$

Esperamos que lo que encontramos en el conjunto de entrenamiento sea cercano al error general, es decir, $L_{m}(\theta^{s}) \approx L_{\Pi}\ (\theta^{s}) = \int \int  \mathscr{l}^{(j)}(\hat{y}\ (\theta),y) \ \Pi(x,y) \ \text{dx}\text{dy}$.

Cuando $(x^{(j)},y^{(j)}) \sim^{iid} \tilde{\Pi}(x,y)$ va a ser erróneo $\tilde{\Pi} \neq \Pi$

### Estrategia de mitigación

* Regulación
* Dropout

### 1.1 Variación en covariables

Suponemos que $\Pi(x,y) = \Pi(y|x)\ \Pi(x)$ y además suponemos que $\tilde{\Pi}(x,y) \neq \Pi(x,y)$,

donde:

* $\tilde{\Pi}(x,y)$ es la distribución de la población,
* $\Pi(x,y)$ es la distribución que genera los datos de entrenamiento.

Esta diferencia en las distribuciones se debe a que $\tilde{\Pi}(x,y) = \tilde{\Pi}(x)\ \Pi(y|x)$, lo que implica que $\tilde{\Pi}(x) \neq \Pi(x)$. $\Pi(y|x)$ establece como generan las etiquetas de los objetos.

Ejemplos:

##### Clasificación de imágenes

* Entrenamiento: conjunto de imágenes reales
* Prueba: Conjunto de imágenes de caricatura

##### Ventas en línea

* Entrenar: conjunto de datos de México
* Prueba: Pronosticar las ventas en Europa

##### Reconocimiento de voz

* Entrenamiento: generar texto automatizado entrenado con gente de la CDMX
* Prueba: producto será utilizado en MTY

$$\$$

En esto contexto estaríamos evaluando una función de pérdida

${}^{(1)}$ $\int \int \ \mathscr{l}(h(x),y) \ \Pi(x)\ \Pi(y|x) \ \text{dxdy} \neq \int \int \mathscr{l}(h(x),y)\  \tilde{\Pi}(x)\ \Pi(y|x) \ \text{dxdy}$ ${}^{(2)}$

Se puede reescribir de la siguiente manera:

${}^{(2)}$ $= \int \int \  \mathscr{l}(h(x),y) \  \frac{\tilde{\Pi}(x)}{\Pi(x)}\ \Pi(x)\ \Pi(y|x) \text{dxdy}$

Definimos  a $\alpha(x) = \frac{\tilde{\Pi}(x)}{\Pi(x)}$

 Sí $\alpha(x) =1$, la pérdida en entrenamiento sería igual que la pérdida de validación (prueba).

 Sí $\alpha(x) \neq 1$

${}^{(2)} \Rightarrow $ $(x^{(i)},y^{(i)}) \sim^{iid} \Pi(x)\ \Pi(y|x)$.

${}^{(2)} \approx \frac{1}{n} \sum_{i=1}^{n} \alpha(x^{(i)})\ \mathscr{l}(h(x^{(i)}),y^{(i)})$ (aproximación Monte Carlo).

La mejor forma es encontrar la función $h(x^{(i)})$ que minimice la función de pérdida con un nuevo ponderador $\alpha$.

Problema: solo tenemos observaciones  y no conocemos $\frac{\tilde{\Pi}}{\Pi}$.

Solución: Entrenar un clasificador que distinga entre $\tilde{\Pi}$ y $\Pi$.

$   \tilde{y} = \begin{cases} 
      1 & \text{si x es un elemento del conjunto de validación} \\
      -1 & \text{si x es un elemento del conjunto de entrenamiento}
   \end{cases}
 $
 
 Con el clasificador binario esperamos que emita probabilidades.
 
 $ P(\tilde{y}=1|x) = \frac{\tilde{\Pi}(x)}{\tilde{\Pi}(x)\ +\ \Pi(x)}$.
 
 De tal forma que si tenemos un clasificador que emita estas probabilidades, los pesos estarán dados de la siguiente forma:
 
 $\alpha(x) = \frac{\tilde{\Pi}(x)}{\Pi(x)} = \frac{P(\tilde{y}=1|x)}{P(\tilde{y}=-1|x)}$.
 
 Podemos softmax para dos etiquetas = regresión logística, dada por la siguiente ecuación: 
 
 $ P(\tilde{y}=1|x) = \frac{exp(O_{1}(x))}{exp(O_{1}(x)) + exp(O_{2}(x))} = \frac{1}{exp(-\tilde{h}(x))}$.
 
 Con $\tilde{h}_{i}(x) = O_{i}(x)-O_{2}(x)$ y $\tilde{h} = \hat{h}_{i}$.
 
 $\hat{\alpha}(x) = exp(\tilde{h}(x))$.
 
 De tal forma que ya tenemos un clasificador que distingue entre datos de entrenamiento y datos de clasificación, podemos ponderar las observaciones. 
 
 Para posteriormente entrenar el modelo $\Rightarrow L_{n}(\theta) = \frac{1}{n} \sum_{i=1}^{n} \hat{\alpha}(x^{(i)}) \ \mathscr{l}(h(x^{(i)}, \theta),y^{(i)}) $.
 
 Minimizamos para encontrar $\theta^{*}$ que tenga la mejor relación entre $x$ y $y$, considerando los pesos $\hat{\alpha}(x^{(i)})$.
 
 Supuesto: $x$ como una muestra de $\Pi$ también es una muestra aleatoria válida de $\tilde{\Pi}$, es decir, no hay una diferencia fundamental entre las distribuciones.
 
 Si tenemos una observación $(k \in {1,...,n})$ tal que $\Pi(x^{(k)}) > 0$ y $\tilde{\Pi}(x^{(k)})=0$ $ \Rightarrow \alpha(x^{(k)}) = 0$.
 
 $L_{n}(h) \rightarrow L_{<n}(h) = \frac{1}{n} \sum_{i=1}^{m} \ \alpha(x^{(i)})\ \mathscr{l}(...) \ m<n $
 
 Es decir, hay elementos que no son representativos de esta relación.
 
 
 ### 1.2 Variación en las etiquetas
 
 Asumimos que $\Pi(x,y) = \Pi(y)\ \Pi(x|y)$ y que $\Pi(y) \neq \tilde{\Pi}(y)$.
 
 Asumimos que $\Pi(x|y)$ se mantiene igual, mismas características.
 
 Ejemplo: clasificación con k etiquetas
 
 $\Pi(y) \rightarrow \Pi_{y} \in \mathbb{R}^{k}$
 
 * Diagnóstico médicos
     - Entrenamiento: con pocos pacientes
     - Validación: periodo de contagio masivo (proporción es mucho mayor que cuando se entrena)
     
     
 * Reconocimiento de diálogos
     - Entrenamiento: durante campaña electoral
     - Validación: fuera de campaña electoral (diferente proporción de palabras)


$ \int \int \mathscr{l}(h(x), y) \ \tilde{\Pi}(y) \ \Pi(x|y) \ \text{dxdy}$

$= \int \int \mathscr{l}(h(x), y) \ \frac{\tilde{\Pi}(y)}{\Pi(y)} \ \Pi(y) \  \Pi(x|y) \ \text{dxdy}$

donde $\frac{\tilde{\Pi}(y)}{\Pi(y)}$ son los pesos y $\Pi(y) \  \Pi(x|y) $ es la distribución de entrenamiento.

Corrección: algoritmo de corrección  espectral.

La idea es: 

$(i)$ Se entrena un clasificador y se obtiene la matriz de confusión en un conjunto ajeno al de entrenamiento.


| Entrenamiento | Validación |
|---------------|------------|
| | |

$(ii)$ Utilizamos el clasificador en el conjunto de prueba. Calculamos el $\%$ de las etiquetas $\mu \in \mathbb{R}^{t}$, $c\tilde{\Pi} = \mu$ y se resuelve el sistema $\tilde{\Pi} \Rightarrow \alpha_{i} = \frac{\tilde{\Pi}_{i}}{Pi_{i}}$